In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [3]:
a_path = Path("Resources/world_population_data.csv")

In [5]:
b_path = Path("Resources/Production_Crops_Livestock_E_All_Data.csv")

In [9]:
world_population = pd.read_csv(a_path)

In [13]:
agri_data = pd.read_csv(b_path, encoding='ISO-8859-1', low_memory = False)


In [15]:
agri_data.info

<bound method DataFrame.info of        Area Code Area Code (M49)                                     Area  \
0              2            '004                              Afghanistan   
1              2            '004                              Afghanistan   
2              2            '004                              Afghanistan   
3              2            '004                              Afghanistan   
4              2            '004                              Afghanistan   
...          ...             ...                                      ...   
79601       5817            '902  Net Food Importing Developing Countries   
79602       5817            '902  Net Food Importing Developing Countries   
79603       5817            '902  Net Food Importing Developing Countries   
79604       5817            '902  Net Food Importing Developing Countries   
79605       5817            '902  Net Food Importing Developing Countries   

       Item Code Item Code (CPC)  \
0      

In [15]:
agri_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79606 entries, 0 to 79605
Columns: 195 entries, Area Code to Y2022N
dtypes: float64(62), int64(3), object(130)
memory usage: 118.4+ MB


In [21]:
agri_data.columns

Index(['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item Code (CPC)',
       'Item', 'Element Code', 'Element', 'Unit', 'Y1961',
       ...
       'Y2019N', 'Y2020', 'Y2020F', 'Y2020N', 'Y2021', 'Y2021F', 'Y2021N',
       'Y2022', 'Y2022F', 'Y2022N'],
      dtype='object', length=195)

In [23]:
pd.set_option('display.max_columns', None)

In [61]:
agri_data["Item"].unique()


array(['Almonds, in shell',
       'Anise, badian, coriander, cumin, caraway, fennel and juniper berries, raw',
       'Apples', 'Apricots', 'Asses', 'Barley',
       'Butter and ghee of sheep milk', 'Butter of cow milk',
       'Buttermilk, dry', 'Camels', 'Cantaloupes and other melons',
       'Cattle', 'Cattle fat, unrendered',
       'Cheese from milk of goats, fresh or processed',
       'Cheese from milk of sheep, fresh or processed', 'Chickens',
       'Cotton lint, ginned', 'Cotton seed', 'Cottonseed oil',
       'Edible offal of cattle, fresh, chilled or frozen',
       'Edible offal of goat, fresh, chilled or frozen',
       'Edible offal of sheep, fresh, chilled or frozen',
       'Edible offals of camels and other camelids, fresh, chilled or frozen',
       'Fat of camels', 'Figs', 'Game meat, fresh, chilled or frozen',
       'Goat fat, unrendered', 'Goats', 'Grapes',
       'Hen eggs in shell, fresh', 'Horses', 'Linseed', 'Maize (corn)',
       'Meat of camels, fresh or c

In [31]:
pd.set_option('display.max_rows', 100)